In [175]:
import numpy as np
import codecs
import regex as re
import spacy
import en_core_web_sm
from collections import defaultdict
from pylab import random
import pandas as pd
from scipy.stats.stats import pearsonr

 N - количество документов
 M - длина словаря слов
 К - кол-во тематик

In [6]:
def stepE():
    for i in range(N):
        for j in range(M):
            denominator = 0
            for k in range(K):
                p[i, j, k] = fi[j, k] * theta[k, i]
                denominator += p[i, j, k]
            if denominator == 0:
                for k in range(0, K):
                    p[i, j, k] = 0
            else:
                for k in range(0, K):
                    p[i, j, k] /= denominator

In [76]:
def stepM():
    # update fi
    for k in range(0, K):
        denominator = 0
        for j in range(0, M):
            fi[j, k] = 0
            for i in range(0, N):
                fi[j, k] += X[i, j] * p[i, j, k]
            denominator += fi[j, k]
        if denominator == 0:
            for j in range(0, M):
                fi[j, k] = 1.0 / M
        else:
            for j in range(0, M):
                fi[j, k] /= denominator
    # update theta
    for i in range(0, N):
        for k in range(0, K):
            theta[k,i] = 0
            denominator = 0
            for j in range(0, M):
                theta[k,i] += X[i, j] * p[i, j, k]
                denominator += X[i,j];
            if denominator == 0:
                theta[k,i] = 1.0 / K
            else:
                theta[k,i] /= denominator

In [125]:
file = codecs.open('datasets2.txt', 'r', 'utf-8')
documents = [document.strip() for document in file] 
file.close()

In [126]:
file = codecs.open('sw.txt', 'r', 'utf-8')
stopwords = [line.strip() for line in file] 
file.close()

In [181]:
pathCV = 'example3/' #Computer vision articles
texts = []
#получаем каждый документ и создаем корпус
for i in range(10):
    fileObj = codecs.open( pathCV + str(i)+'.txt', "r", "utf_8_sig" )
    text = fileObj.read() # или читайте по строке
    texts.append(text)
    fileObj.close()

In [127]:
def remove_punctuation(text):
    return re.sub(r'\p{P}+', "", text)
def change_num(text):
    return re.sub(r'[\d]+', "1", text)  

In [128]:
nlp = en_core_web_sm.load()
def lemmatization(text):
    tx = change_num(remove_punctuation(text.lower()))
    doc = nlp(tx)
    return [token.lemma_ for token in doc if token.lemma_ not in stopwords]

In [182]:
X_lem = [lemmatization(text) for text in texts]

In [183]:
N = len(X_lem)
N

10

In [184]:
tokens = defaultdict(int)#
count_words =[]
for doc in X_lem:
    count_word = defaultdict(int)
    for word in doc:
        tokens[word] += 1
        count_word[word] = +1
    count_words.append(count_word)
M = len(tokens)
M

3761

In [185]:
words = []
for key in tokens.keys():
    words.append(key)

In [186]:
X = np.zeros([N, M])

In [187]:
for j,word in enumerate(words):
    for i,doc in enumerate(X_lem):
        X[i,j] = count_words[i][word]

In [188]:
X.shape

(10, 3761)

In [206]:
K = 20    # number of topic
maxIteration = 20
threshold = 10.0
topicWordsNum = 10

In [207]:

fi = random([M, K])
theta = random([K, N])
p = np.zeros([N, M, K])


In [208]:
def initializeParameters():
    for i in range(0, N):
        normalization = sum(theta[:, i])
        for j in range(0, K):
            theta[j, i] /= normalization

    for i in range(0, K):
        normalization = sum(fi[:, i])
        for j in range(0, M):
            fi[j, i] /= normalization

In [209]:
def LogLikelihood():
    loglikelihood = 0
    for i in range(0, N):
        for j in range(0, M):
            tmp = 0
            for k in range(0, K):
                tmp += fi[j, k] * theta[k, i]
            if tmp > 0:
                loglikelihood += X[i, j] * np.log(tmp)
    return loglikelihood

In [211]:
fi

array([[0.68542732, 0.4922167 , 0.26394482, ..., 0.54459375, 0.8138456 ,
        0.36531612],
       [0.45936685, 0.71825421, 0.78572027, ..., 0.0989575 , 0.18335985,
        0.79688551],
       [0.23672602, 0.64880209, 0.44491489, ..., 0.22277439, 0.02264028,
        0.28760738],
       ...,
       [0.42445104, 0.97934239, 0.7972429 , ..., 0.60461378, 0.78930225,
        0.56032957],
       [0.1814033 , 0.44007232, 0.04433484, ..., 0.37721344, 0.38509809,
        0.24969651],
       [0.48519889, 0.91684516, 0.2494058 , ..., 0.9056238 , 0.12780078,
        0.50029401]])

In [212]:
initializeParameters()

In [213]:
fi

array([[3.62268537e-04, 2.62086261e-04, 1.41716571e-04, ...,
        2.96284512e-04, 4.33684911e-04, 1.95489281e-04],
       [2.42788914e-04, 3.82442451e-04, 4.21866898e-04, ...,
        5.38375139e-05, 9.77094438e-05, 4.26432256e-04],
       [1.25116679e-04, 3.45461896e-04, 2.38882552e-04, ...,
        1.21199706e-04, 1.20646344e-05, 1.53905500e-04],
       ...,
       [2.24334884e-04, 5.21461761e-04, 4.28053598e-04, ...,
        3.28938223e-04, 4.20606161e-04, 2.99845586e-04],
       [9.58769895e-05, 2.34321409e-04, 2.38041461e-05, ...,
        2.05221785e-04, 2.05212424e-04, 1.33618499e-04],
       [2.56441911e-04, 4.88184414e-04, 1.33910318e-04, ...,
        4.92701777e-04, 6.81029274e-05, 2.67719141e-04]])

In [196]:
import time

In [214]:
oldLoglikelihood = 1
newLoglikelihood = 1
for i in range(0, maxIteration):
    stepE()
    stepM()
    newLoglikelihood = LogLikelihood()
    print("[", time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), "] ", i+1, " iteration  ", str(newLoglikelihood))
    if(oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < threshold):
        break
    oldLoglikelihood = newLoglikelihood

[ 2020-05-30 00:39:26 ]  1  iteration   -45456.855617596986
[ 2020-05-30 00:39:32 ]  2  iteration   -44734.068852379714
[ 2020-05-30 00:39:37 ]  3  iteration   -43625.7708957977
[ 2020-05-30 00:39:42 ]  4  iteration   -42148.41588392831
[ 2020-05-30 00:39:47 ]  5  iteration   -40613.81751249084
[ 2020-05-30 00:39:53 ]  6  iteration   -39402.59314825921
[ 2020-05-30 00:39:58 ]  7  iteration   -38603.67059116329
[ 2020-05-30 00:40:03 ]  8  iteration   -38136.39101040347
[ 2020-05-30 00:40:08 ]  9  iteration   -37889.8364458977
[ 2020-05-30 00:40:14 ]  10  iteration   -37759.53372470167
[ 2020-05-30 00:40:19 ]  11  iteration   -37680.345769979234
[ 2020-05-30 00:40:24 ]  12  iteration   -37621.95245226537
[ 2020-05-30 00:40:29 ]  13  iteration   -37562.226523812235
[ 2020-05-30 00:40:35 ]  14  iteration   -37479.415076305275
[ 2020-05-30 00:40:40 ]  15  iteration   -37374.27558784685
[ 2020-05-30 00:40:45 ]  16  iteration   -37292.19691217964
[ 2020-05-30 00:40:50 ]  17  iteration   -3724

In [215]:
fi.shape

(3761, 20)

In [216]:
tops_w =[]
for i in range(0, K):
    topicword = []
    ids = fi[:, i].argsort()
    for j in ids:
        topicword.insert(0, words[j])
    tmp = ''
    for word in topicword[0:min(topicWordsNum, len(topicword))]:
        tmp += word + ' '
    tops_w.append(tmp)

In [217]:
tops_w

['situ conclude promote interaction reduction trigger strong induction comprise wellensiek ',
 'wine birth party multicellular improve herbal cornflower generation oxygen actively ',
 'explain oak wildflower botanist reproductive solanum mountain dispersal greek pond ',
 'lake freshwater atmosphere spell lemnaceae unrelated store size wolffia burn ',
 'pattern occur molecular lily floral pollen insect female nectar wind ',
 'represent wide success taxon define change environment fashion lack wing ',
 'feed blossom cluster family sunflow tiny central top attractive citriodora ',
 'spiny surround approach gardens fall grassland typical bed acre meadow ',
 'choose mix create inspire couple learn design period nature thistle ',
 'biennial establish free typical size endosporic start chiropterophily common salverform ',
 'simply identification depend dry density provide mark condition characterize shortlived ',
 'visit foster lend nourish behold march predate word pond advantage ',
 'primor

In [165]:
data = pd.read_csv('wordsim_similarity_goldstandard.txt',sep='\t', names=['word1','word2','sim'])

In [167]:
word1 = data['word1'].values
word2 = data['word2'].values
scores = data['sim'].values

In [218]:
indexs = []
for k,w in enumerate(zip(word1,word2)):
    if (w[0] in words) and (w[1] in words):
        indexs.append(k)

In [219]:
len(indexs)

28

In [220]:
def cos(A,B):
    SumA = 0 
    SumB = 0
    SumAB = 0
    for i in range(len(A)):
        SumAB += A[i]*B[i]
        SumA += A[i]**2
        SumB += B[i]**2
    return SumAB/(np.sqrt(SumA)*np.sqrt(SumB))

def jac(A,B):

    SumA = 0 
    SumB = 0
    for i in range(len(A)):
        SumA += min(A[i],B[i])
        SumB += max(A[i],B[i])
    return SumA/SumB

def KL(A,B):
    Sum = 0
    for i in range (len(A)):
        if (A[i] != 0) and B[i] != 0:
            Sum += A[i] * np.log(A[i]/B[i])
    return Sum

def JS(A,B):
    AB = [(u[0]+u[1])/2 for u in zip(A,B)]
    return (KL(A,AB)+KL(B,AB))/2

In [221]:
C =[]
cos_dist = []
jac_dist = []
KL_dist = []
JS_dist = []
SC = []
for j in indexs:
    index_w1 = words.index(word1[j])
    index_w2 = words.index(word2[j])
    A = fi[index_w1]
    B = fi[index_w2]
    
    cos_dist.append(cos(A,B))
    jac_dist.append(jac(A,B))
    KL_dist.append(KL(A,B))
    JS_dist.append(JS(A,B))
    SC.append(scores[j])
    
C.append((pearsonr(cos_dist,SC)[0],pearsonr(jac_dist,SC)[0],pearsonr(KL_dist,SC)[0],pearsonr(JS_dist,SC)[0]))
print(C)   

[(0.029105132110236312, -0.021953617525467078, -0.31042686098720423, 0.06050863208558216)]
